# This notebook provides the commands needed to read in an NWB file with raw data taken from a trodes .rec file and to copy the relevant metadata into a DataJoint Schema designed to mimic NWB structure

## The following steps must be completed before running this notebook:
### 1) Set up access to a DataJoint database. See https://tutorials.datajoint.io/
### 2) Install pynwb. 
    See https://pynwb.readthedocs.io/en/stable/getting_started.html#installation
### 3) Install the ndx-fl-novela package. 
     conda install -c novelakrk ndx-fl-novela
### 4) Download the example franklab NWB file from DropBox. 
    https://www.dropbox.com/to_be_updated

#### Make sure we're in the franklabnwb/franklabnwb directory. 
#### Eventually this will be unnecessary because the package will properly installed

In [1]:
cd ../schema

/Users/loren/Src/NWB/nwb_datajoint/schema


#### Load all of the relevant modules

In [10]:
%env DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
%load_ext autoreload
%autoreload 2


import pynwb
import os

#DataJoint and DataJoint schema
import datajoint as dj

dj.config["enable_python_native_blobs"] = True
dj.config['stores'] = {
  'local': dict(  # 'regular' external storage for this pipeline
                protocol='file',
                location='/data/nwb_builder_test_data/dj/',
                stage='/data/nwb_builder_test_data/dj/'),
  'local-filtered': dict( # 'local_filtered' storage filtered data in hdf5 files
                protocol='file',
                location='/data/nwb_builder_test_data/dj/filtered/',
                stage='/data/nwb_builder_test_data/dj/filtered/')
}

import common_session
import common_ephys
import common_device
import common_lab
import common_task
import common_behav
import common_interval
import common_region
import common_filter
import nwb_dj
import ndx_franklab_novela.probe

import spikeinterface as si
import spikeextractors as se

env: DJ_SUPPORT_FILEPATH_MANAGEMENT=TRUE
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Set the nwb file name and the name of the probe file to create from DataJoint

In [30]:
nwb_file_name = '/data/nwb_builder_test_data/dj/beans20190718_pp.nwb'
probe_file_name = '/data/nwb_builder_test_data/test.prb'

In [31]:
common_ephys.ElectrodeSortingInfo().initialize(nwb_file_name)
common_ephys.ElectrodeSortingInfo().set_group_by_shank(nwb_file_name)

In [32]:
common_ephys.ElectrodeSortingInfo().write_prb(nwb_file_name, probe_file_name)

In [33]:
recording = se.NwbRecordingExtractor(nwb_file_name, electrical_series_name='e-series')



In [34]:
recording = recording.load_probe_file(probe_file_name)


#p = si.widgets.plot_electrode_geometry(recording)

In [35]:
common_ephys.SpikeSorter().insert_from_spikeinterface()
common_ephys.SpikeSorterParameters().insert_from_spikeinterface()



About to delete:
`common_ephys`.`spike_sorter_parameters`: 10 items
`common_ephys`.`spike_sorter`: 10 items


Proceed? [yes, No]:  yes


Committed.


In [36]:
p = (common_ephys.SpikeSorterParameters() & {'sorter_name': 'mountainsort4'}).fetch1()
print(p)
param = p['parameter_dict']
param['adjacency_radius'] = 100
param['curation'] = True
param['num_workers'] = 1

{'sorter_name': 'mountainsort4', 'parameter_set_name': 'default', 'parameter_dict': {'detect_sign': -1, 'adjacency_radius': -1, 'freq_min': 300, 'freq_max': 6000, 'filter': True, 'whiten': True, 'curation': False, 'num_workers': None, 'clip_size': 50, 'detect_threshold': 3, 'detect_interval': 10, 'noise_overlap_threshold': 0.15}}


In [38]:
first_shank_electrodes = (common_ephys.ElectrodeSortingInfo() & {'nwb_file_name' : nwb_file_name} & {'sort_group' : 0}).fetch('electrode_id')
first_shank_electrodes

array([ 0,  1,  3,  4,  5,  7,  8,  9, 11, 12, 13, 15, 16, 17, 19, 20, 21,
       23, 24, 25, 27, 28, 29, 31])

In [41]:
first_shank_recording = se.SubRecordingExtractor(recording, channel_ids=first_shank_electrodes)

In [42]:
first_shank_sort = si.sorters.run_mountainsort4(recording=first_shank_recording, **param, grouping_property='group', output_folder='/data/nwb_builder_test_data/tmp')

KeyboardInterrupt: 